# Assignment -  Enhanced RAG Application

In this assignment, we are going to expand our RAG application to process PDF docuemnts.

First, we're going to import all the libraries needed:

In [9]:
from aimakerspace.pdf_utils import PDFLoader
from aimakerspace.text_utils import CharacterTextSplitter, TextFileLoader
from aimakerspace.vectordatabase import VectorDatabase
import asyncio
import os
import openai
from getpass import getpass
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

import nest_asyncio
nest_asyncio.apply()

Then, we will request the OpenAI key:

In [10]:
openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Now, we will create the necessary objects to operate our embeddings, database, and chat

In [11]:
text_splitter = CharacterTextSplitter()
vector_db = VectorDatabase()
chat_openai = ChatOpenAI()

Now, we proceed to load our PDF document, chunk it and store its embeddings:

In [12]:
pdf_loader = PDFLoader("data/us_youth_soccer_coaching_manual.pdf")
pdf_loader.load_documents()
documents = pdf_loader.documents
split_documents = text_splitter.split_texts(documents)

vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

Now, we will proceed to create our RAG prompts:

In [13]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "simple",
        "response_length": "short"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

... and our pipeline...

In [14]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

Now, with all these pieces in place, we can proceed to query our PDF document:

In [15]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What are the differences between coaching kids five to six years old and kids seven to eight years old?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: The differences between coaching kids five to six years old and kids seven to eight years old primarily revolve around their developmental stages and how they engage with the game.

For the U-6 age group (children aged approximately four to seven), the focus is largely on play. According to Source 1, during this period, children are absorbed in games of their own devising, indicating that coaching should emphasize a child-centered learning environment. The developmental growth can vary significantly among players in this age group, making it important for coaches to understand individual differences. Activities should encourage individual ball work as well as cooperative play, allowing for both egocentric and collaborative interactions.

In contrast, for the seven to eight year old age group, as noted in Source 3, players begin to develop a greater awareness beyond their own needs, actively looking to pass to teammates. This age marks a shift towards more cooperative play, as